In [ ]:
# 测试读取数据

In [ ]:
# 数据同行运算

In [43]:
import pandas as pd

#风力数据
df = pd.DataFrame({'A': [40336, 2125, 3026, 107158, 630050, 800270, 70045, 900012, 600045]})
df['sped'] = df.apply(lambda x: (x.A - int(str(int(x.A))[-3:]))/10000, axis = 1)
df['dir'] = df.apply(lambda x: int(str(int(x.A))[-3:]), axis = 1)
df

,A,sped,dir
0,40336,4.0,336
1,2125,0.2,125
2,3026,0.3,26
3,107158,10.7,158
4,630050,63.0,50
5,800270,80.0,270
6,70045,7.0,45
7,900012,90.0,12
8,600045,60.0,45


In [52]:
# 行运算
import pandas as pd

def data(x):
    return x.A - x.B
#风力数据
df = pd.DataFrame({'A': [40336, 2125, 3026, 107158, 630050, 800270, 70045, 900012, 600045],
                  'B': [40336, 2125, 3026, 107158, 630050, 800270, 70045, 900012, 600045]})
df['sped'] = df.apply(data, axis = 1)
df

,A,B,sped
0,40336,40336,0
1,2125,2125,0
2,3026,3026,0
3,107158,107158,0
4,630050,630050,0
5,800270,800270,0
6,70045,70045,0
7,900012,900012,0
8,600045,600045,0


In [72]:
# 列运算
import pandas as pd

def data(x):
    #x.drop(3,inplace=True)
    print(x.name)
    x.loc[3] = 666
#     for i in range(len(x)):
#         if x[i] ==0:
#             x[i]=1
    da = sum(x)
    return da
#风力数据
df = pd.DataFrame({'A': [1, 2, 3, 0, 0, 0, 0, 0, 0],
                  'B': [1, 1, 1, 1, 1, 1, 1, 1, 1]})
df.apply(data, axis = 0).A


A
B


672

In [ ]:
# 小时数据更新

In [33]:
import datetime as dtt
now_times = "2023-07-18 13:15:00"
now = dtt.datetime.strptime(now_times,"%Y-%m-%d %H:%M:%S")
mins = now.minute
offset =dtt.timedelta(minutes=-mins)
hours = now + offset
 
start_time = hours.strftime('%Y-%m-%d %H:%M:%S')
end_time = now.strftime('%Y-%m-%d %H:%M:%S')

## 天擎

datetime.datetime(2023, 7, 18, 13, 0)

# 数据增量同步

In [89]:
%%time
import pandas as pd
data = pd.read_csv("downfile/min.csv")
data = data.replace(999999.0, np.nan)
sql_data = data[['Station_Id_C','Cnty','City','Province','Station_levl','Station_Name','Town','Alti','Lat','Lon','PRE','WIN_S_Gust_Max','WIN_D_Gust_Max','VIS_HOR_1MI','TEM']]
def comput_data(data,delay):
    grouped = data.groupby('Station_Id_C')
    for name,group in grouped:
        single = group
        # 单站处理
#         single['tmin'] = single['TEM']
#         single[single['WIN_S_Gust_Max']==999999]=-999999
#         max_time = single['Datetime'].max   
#         #station = pd.DataFrame({'current_time':single['Datetime'].max,'rain':sum(single['PRE']),'tmax':single['TEM'].max,'tmin':single['TEM'].min,'view':single['VIS_HOR_1MI'].min,'wind':single['VIS_HOR_1MI'].min})
#         # 计算聚合值
#     return single

delay = 24
single = comput_data(data,delay)
single


CPU times: user 373 ms, sys: 10.1 ms, total: 383 ms
Wall time: 393 ms


# 更新逐小时数据

In [ ]:
# 

In [29]:
%%time
import pymysql
import pymssql 
import numpy as np
import pandas as pd
import pickle
import redis
import datetime as dtt
import json
from datetime import timezone



class station_zdz:
    def __init__(self):
        self.rs = redis.Redis(host='127.0.0.1', port=6379)
        self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
    def sql_wind(self):
        start_times = '2023-07-18 12:00:00'
        end_times = '2023-07-18 12:59:00'
        sql = """select max(City) as City,max(Cnty) as Cnty, Station_Id_C , max(Province) as Province, max(Station_levl) as Station_levl,
        max(Station_Name) as Station_Name, max(Town) as Town, max(Alti) as Alti, max(Lat) as Lat,max(Lon) as Lon, max(WIN_S_Gust_Max*10000 + WIN_D_Gust_Max) as wind
        from table_station_min 
        where Datetime between '{start_times}' and '{end_times}' and WIN_S_Gust_Max<5000 and WIN_D_Gust_Max<5000
        group by Station_Id_C""" 
        rsql = sql.format(start_times=start_times,end_times=end_times)
        data = pd.read_sql(rsql, con=self.conn)
        sql_data = data[['Station_Id_C','Cnty','City','Province','Station_levl','Station_Name','Town','Alti','Lat','Lon','wind']]
        sql_data['Datetime'] = start_times
        datalist = sql_data.to_json(orient='values')
        sql_list = json.loads(datalist)
        conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
        cursor = conn.cursor()
        insql = """insert into table_station_hour 
        (Station_Id_C,Cnty,City,Province,Station_levl,Station_Name,Town,Alti,Lat,Lon,wind,Datetime) 
        values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        on duplicate key update
        wind = values(wind),
        Station_Id_C = values(Station_Id_C),
        Cnty = values(Cnty),
        City = values(City),
        Province = values(Province),
        Station_levl = values(Station_levl),
        Station_Name = values(Station_Name),
        Town = values(Town),
        Alti = values(Alti),
        Station_Name = values(Station_Name),
        Lat = values(Lat),
        Lon = values(Lon),
        Datetime = values(Datetime)"""
        cursor.executemany(insql,sql_list)
        cursor.close()
        cursor = conn.cursor()
        conn.commit()
        conn.close()
    def sql_temp(self):
        start_times = '2023-07-18 12:00:00'
        end_times = '2023-07-18 12:59:00'
        sql = """select max(City) as City,max(Cnty) as Cnty, Station_Id_C , max(Province) as Province, max(Station_levl) as Station_levl,
        max(Station_Name) as Station_Name, max(Town) as Town, max(Alti) as Alti, max(Lat) as Lat,max(Lon) as Lon, max(TEM) as tmax,min(TEM) as tmin
        from table_station_min 
        where Datetime between '{start_times}' and '{end_times}' and TEM<5000 
        group by Station_Id_C""" 
        rsql = sql.format(start_times=start_times,end_times=end_times)
        data = pd.read_sql(rsql, con=self.conn)
        sql_data = data[['Station_Id_C','Cnty','City','Province','Station_levl','Station_Name','Town','Alti','Lat','Lon','tmax','tmin']]
        sql_data['Datetime'] = start_times
        datalist = sql_data.to_json(orient='values')
        sql_list = json.loads(datalist)
        conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
        cursor = conn.cursor()
        insql = """insert into table_station_hour 
        (Station_Id_C,Cnty,City,Province,Station_levl,Station_Name,Town,Alti,Lat,Lon,tmax,tmin,Datetime) 
        values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        on duplicate key update
        tmax = values(tmax),
        tmin = values(tmin),
        Station_Id_C = values(Station_Id_C),
        Cnty = values(Cnty),
        City = values(City),
        Province = values(Province),
        Station_levl = values(Station_levl),
        Station_Name = values(Station_Name),
        Town = values(Town),
        Alti = values(Alti),
        Station_Name = values(Station_Name),
        Lat = values(Lat),
        Lon = values(Lon),
        Datetime = values(Datetime)"""
        cursor.executemany(insql,sql_list)
        cursor.close()
        cursor = conn.cursor()
        conn.commit()
        conn.close()
    def sql_rain(self):
        start_times = '2023-07-17 13:00:00'
        end_times = '2023-07-17 13:59:00'
        sql = """select max(City) as City,max(Cnty) as Cnty, Station_Id_C , max(Province) as Province, max(Station_levl) as Station_levl,
        max(Station_Name) as Station_Name, max(Town) as Town, max(Alti) as Alti, max(Lat) as Lat,max(Lon) as Lon, sum(PRE) as rain , max(PRE_1h) as rain_max
        from table_station_min 
        where Datetime between '{start_times}' and '{end_times}' and PRE<5000  
        group by Station_Id_C""" 
        rsql = sql.format(start_times=start_times,end_times=end_times)
        data = pd.read_sql(rsql, con=self.conn)
        sql_data = data[['Station_Id_C','Cnty','City','Province','Station_levl','Station_Name','Town','Alti','Lat','Lon','rain','rain_max']]
        sql_data['Datetime'] = start_times
        datalist = sql_data.to_json(orient='values')
        sql_list = json.loads(datalist)
        conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
        cursor = conn.cursor()
        insql = """insert into table_station_hour 
        (Station_Id_C,Cnty,City,Province,Station_levl,Station_Name,Town,Alti,Lat,Lon,rain,rain_max,Datetime) 
        values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        on duplicate key update
        rain = values(rain),
        rain_max = values(rain_max),
        Station_Id_C = values(Station_Id_C),
        Cnty = values(Cnty),
        City = values(City),
        Province = values(Province),
        Station_levl = values(Station_levl),
        Station_Name = values(Station_Name),
        Town = values(Town),
        Alti = values(Alti),
        Station_Name = values(Station_Name),
        Lat = values(Lat),
        Lon = values(Lon),
        Datetime = values(Datetime)"""
        cursor.executemany(insql,sql_list)
        cursor.close()
        cursor = conn.cursor()
        conn.commit()
        conn.close()
    def sql_view(self):
        start_times = '2023-07-18 12:00:00'
        end_times = '2023-07-18 12:59:00'
        sql = """select max(City) as City,max(Cnty) as Cnty, Station_Id_C , max(Province) as Province,max(Station_levl) as Station_levl,
        max(Station_Name) as Station_Name, max(Town) as Town, max(Alti) as Alti, max(Lat) as Lat,max(Lon) as Lon, min(VIS_HOR_1MI) as view
        from table_station_min 
        where Datetime between '{start_times}' and '{end_times}' and VIS_HOR_1MI<30000 
        group by Station_Id_C""" 
        rsql = sql.format(start_times=start_times,end_times=end_times)
        data = pd.read_sql(rsql, con=self.conn)
        sql_data = data[['Station_Id_C','Cnty','City','Province','Station_levl','Station_Name','Town','Alti','Lat','Lon','view']]
        sql_data['Datetime'] = start_times
        datalist = sql_data.to_json(orient='values')
        sql_list = json.loads(datalist)
        conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="051219",db="ZJSZDZDB")
        cursor = conn.cursor()
        insql = """insert into table_station_hour
        (Station_Id_C,Cnty,City,Province,Station_levl,Station_Name,Town,Alti,Lat,Lon,view,Datetime) 
        values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        on duplicate key update
        view = values(view),
        Station_Id_C = values(Station_Id_C),
        Cnty = values(Cnty),
        City = values(City),
        Province = values(Province),
        Station_levl = values(Station_levl),
        Station_Name = values(Station_Name),
        Town = values(Town),
        Alti = values(Alti),
        Station_Name = values(Station_Name),
        Lat = values(Lat),
        Lon = values(Lon),
        Datetime = values(Datetime)"""
        cursor.executemany(insql,sql_list)
        cursor.close()
        cursor = conn.cursor()
        conn.commit()
        conn.close()
    

    
worker = station_zdz()
data = worker.sql_rain()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 790 ms, sys: 10.2 ms, total: 800 ms
Wall time: 1.29 s


In [ ]:
# 